Célula 1 - Setup Completo (Lexer + Parser)

In [ ]:
import re
from dataclasses import dataclass
from typing import List, Tuple, Any, Optional

# ===== Definição dos Tokens =====
TOKEN_SPEC = [
    ("NUMBER",   r"\d+(\.\d+)?"),  # Números (inteiros e decimais)
    ("IDENT",    r"[A-Za-z_]\w*"), # Identificadores (letras e sublinhados)
    ("PLUS", r"\+"),                # Operador de soma
    ("MINUS", r"-"),                # Operador de subtração
    ("STAR", r"\*"),                # Operador de multiplicação
    ("SLASH", r"/"),                # Operador de divisão
    ("CARET", r"\^"),               # Operador de potência
    ("LPAREN", r"\("),              # Parêntese esquerdo
    ("RPAREN", r"\)"),              # Parêntese direito
    ("SKIP", r"[ \t\r\n]+"),        # Ignora espaços, tabs e quebras de linha
    ("MISMATCH", r".")              # Para caracteres que não se encaixam
]

# Compilando a expressão regular para o lexer
MASTER_RE = re.compile("|".join(f"(?P<{name}>{regex})" for name, regex in TOKEN_SPEC))

@dataclass
class Token:
    type: str
    value: str
    pos: int

def tokenize(code: str) -> List[Token]:
    tokens = []
    for m in MASTER_RE.finditer(code):
        kind = m.lastgroup
        value = m.group()
        pos = m.start()
        if kind == "SKIP":
            continue
        elif kind == "MISMATCH":
            raise SyntaxError(f"Caractere inesperado '{value}' na posição {pos}")
        tokens.append(Token(kind, value, pos))
    return tokens

# ===== Definição dos Nós da AST (Árvore Sintática) =====
@dataclass
class Num:
    value: float

@dataclass
class Var:
    name: str

@dataclass
class Unary:
    op: str
    expr: Any

@dataclass
class Binary:
    op: str
    left: Any
    right: Any

def ast_str(node: Any, indent: int = 0) -> str:
    pad = "  " * indent
    if isinstance(node, Num):
        return f"{pad}Num({node.value})"
    elif isinstance(node, Var):
        return f"{pad}Var({node.name})"
    elif isinstance(node, Unary):
        return f"{pad}Unary({node.op})\n{ast_str(node.expr, indent + 1)}"
    elif isinstance(node, Binary):
        return (f"{pad}Binary({node.op})\n" +
                f"{ast_str(node.left, indent + 1)}\n" +
                f"{ast_str(node.right, indent + 1)}")
    return f"{pad}{node!r}"

# ===== Parser (Analisador Sintático) =====
class ParseError(Exception): ...

class Parser:
    def __init__(self, tokens: List[Token]):
        self.tokens = tokens
        self.i = 0

    def peek(self, *types) -> Optional[Token]:
        if self.i < len(self.tokens) and self.tokens[self.i].type in types:
            return self.tokens[self.i]
        return None

    def eat(self, *types) -> Token:
        if self.peek(*types):
            t = self.tokens[self.i]
            self.i += 1
            return t
        raise ParseError(f"Esperado um dos tipos {types}, mas obtido {self.tokens[self.i].type}.")

    def parse(self) -> Any:
        node = self.expr()
        if self.i != len(self.tokens):
            raise ParseError("Entrada extra após a expressão.")
        return node

    # Define regras para expressões, termos, potências e unidades
    def expr(self) -> Any:
        node = self.term()
        while self.peek("PLUS", "MINUS"):
            op = self.eat("PLUS", "MINUS").value
            node = Binary(op, node, self.term())
        return node

    def term(self) -> Any:
        node = self.power()
        while self.peek("STAR", "SLASH"):
            op = self.eat("STAR", "SLASH").value
            node = Binary(op, node, self.power())
        return node

    def power(self) -> Any:
        node = self.unary()
        while self.peek("CARET"):
            self.eat("CARET")
            node = Binary("^", node, self.unary())
        return node

    def unary(self) -> Any:
        if self.peek("MINUS"):
            self.eat("MINUS")
            return Unary("-", self.unary())
        return self.primary()

    def primary(self) -> Any:
        if self.peek("NUMBER"):
            tok = self.eat("NUMBER")
            return Num(float(tok.value))
        if self.peek("LPAREN"):
            self.eat("LPAREN")
            node = self.expr()
            self.eat("RPAREN")
            return node
        raise ParseError("Elemento primário inválido.")

# Teste inicial da Tokenização e AST
test_code = "3 + 4 * (2 - 1) ^ 2"
tokens = tokenize(test_code)
print("Tokens:", [(t.type, t.value) for t in tokens])

# Criando o Parser e gerando a AST
parser = Parser(tokens)
ast = parser.parse()
print("AST:", ast_str(ast))

Tokens: [('NUMBER', '3'), ('PLUS', '+'), ('NUMBER', '4'), ('STAR', '*'), ('LPAREN', '('), ('NUMBER', '2'), ('MINUS', '-'), ('NUMBER', '1'), ('RPAREN', ')'), ('CARET', '^'), ('NUMBER', '2')]
AST: Binary(+)
  Num(3.0)
  Binary(*)
    Num(4.0)
    Binary(^)
      Binary(-)
        Num(2.0)
        Num(1.0)
      Num(2.0)


Célula 2 - Visualizando o Pipeline de Tokenização e Análise Sintática

In [ ]:
def run_pipeline(expr: str):
    print(f"\n=== Processando a expressão: {expr}")

    # Tokenização
    tokens = tokenize(expr)
    print("Tokens:", [(t.type, t.value) for t in tokens])

    # Construção da AST
    parser = Parser(tokens)
    ast = parser.parse()
    print("\nÁrvore Sintática (AST):")
    print(ast_str(ast))

    # Você pode adicionar aqui a fase de otimização ou outras análises, conforme necessário
    print("\nA análise léxica foi completada com sucesso!")

# Testando o pipeline com uma expressão matemática simples
run_pipeline("3 + 4 * (2 - 1) ^ 2")


=== Processando a expressão: 3 + 4 * (2 - 1) ^ 2
Tokens: [('NUMBER', '3'), ('PLUS', '+'), ('NUMBER', '4'), ('STAR', '*'), ('LPAREN', '('), ('NUMBER', '2'), ('MINUS', '-'), ('NUMBER', '1'), ('RPAREN', ')'), ('CARET', '^'), ('NUMBER', '2')]

Árvore Sintática (AST):
Binary(+)
  Num(3.0)
  Binary(*)
    Num(4.0)
    Binary(^)
      Binary(-)
        Num(2.0)
        Num(1.0)
      Num(2.0)

A análise léxica foi completada com sucesso!


Célula 3 - Exercício e Análise Passo a Passo

In [ ]:
try:
    while True:
        s = input("\nDigite uma expressão (ENTER para sair): ").strip()
        if not s: break
        run_pipeline(s)
except KeyboardInterrupt:
    pass


Digite uma expressão (ENTER para sair): -3 + 4 * 2

=== Processando a expressão: -3 + 4 * 2
Tokens: [('MINUS', '-'), ('NUMBER', '3'), ('PLUS', '+'), ('NUMBER', '4'), ('STAR', '*'), ('NUMBER', '2')]

Árvore Sintática (AST):
Binary(+)
  Unary(-)
    Num(3.0)
  Binary(*)
    Num(4.0)
    Num(2.0)

A análise léxica foi completada com sucesso!
